<h1>Chapter 10. Implementing Design Patterns with Full-Fledged Functions</h1>

<h2>Practical Example: Reworking a Pattern Strategy</h2>

<h3>Classic Strategy</h3>

The Strategy pattern allows clients to select from a range of interchangeable algorithms during runtime, promoting flexibility and maintainability. It consists of three main components:</br>
**Context**: This class maintains a reference to the current strategy and collaborates with it, providing an interface for accessing strategy algorithms.</br>
**Strategy**: This interface or abstract class defines the contract for all supported algorithms, specifying common methods that concrete strategies must implement.</br>
**Concrete Strategies**: These are individual algorithm implementations that adhere to the Strategy interface, providing specific implementations for each algorithm.

Implementation of the `Order` class using interchangeable discounting strategies. 

In [1]:
from abc import ABC, abstractmethod
from collections.abc import Sequence
from decimal import Decimal
from typing import NamedTuple, Optional


class Customer(NamedTuple):
    name: str
    fidelity: int


class LineItem(NamedTuple):
    product: str
    quantity: int
    price: Decimal

    def total(self) -> Decimal:
        return self.price * self.quantity


class Order(NamedTuple):  # the Context
    customer: Customer
    cart: Sequence[LineItem]
    promotion: Optional['Promotion'] = None

    def total(self) -> Decimal:
        totals = (item.total() for item in self.cart)
        return sum(totals, start=Decimal(0))

    def due(self) -> Decimal:
        if self.promotion is None:
            discount = Decimal(0)
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self):
        return f"<Order total: {self.total():.2f} due: {self.due():.2f}>"


class Promotion(ABC):  # the Strategy: an abstract base class
    @abstractmethod
    def discount(self, order: Order) -> Decimal:
        """Return discount as a possitive dollar amount"""


class FidelityPromo(Promotion):  # first Concrete Strategy
    """5% discount for customers with 1000 or more fidelity points"""

    def discount(self, order: Order) -> Decimal:
        rate = Decimal('0.05')
        if order.customer.fidelity >= 1000:
            return order.total() * rate
        return Decimal(0)


class BulkItemPromo(Promotion):  # second Concrete Strategy
    """10% discount for each LineItem with 20 or more units"""

    def discount(self, order: Order) -> Decimal:
        discount = Decimal(0)
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * Decimal('0.1')
        return discount


class LargeOrderPromo(Promotion):  # third Concrete Strategy
    """7% discount for orders with 10 or more distinct items"""

    def discount(self, order: Order) -> Decimal:
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * Decimal('0.07')
        return Decimal(0)

In [2]:
# Two customers: Joe has 0 loyalty, Ann has 1100
joe = Customer(name='John Doe', fidelity=0)
ann = Customer(name='Ann Smith', fidelity=1100)

# One shopping card with three items
cart = (
    LineItem(product='banana', quantity=4, price=Decimal('.5')),
    LineItem(product='apple', quantity=10, price=Decimal('1.5')),
    LineItem(product='watermelon', quantity=5, price=Decimal(5)),
)

# The banana_cart contains 30 bananas and 10 apples
banana_cart = (
    LineItem(product='banana', quantity=30, price=Decimal('.5')),
    LineItem(product='apple', quantity=10, price=Decimal('1.5')),
)

# There are different items in the long_cart that cost 1.00$ each
long_cart = tuple(
    LineItem(product=str(sku), quantity=1, price=Decimal(1)) for sku in range(10)
)

In [3]:
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [4]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [5]:
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [6]:
Order(ann, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [7]:
Order(joe, long_cart, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [8]:
Order(ann, long_cart, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

<h2>Functionally-Oriented Strategy</h2>

`Order` class, in which the discounting srategies are implemented as functions

In [9]:
from collections.abc import Sequence
from decimal import Decimal
from typing import Callable, NamedTuple, Optional


class Customer(NamedTuple):
    name: str
    fidelity: int


class LineItem(NamedTuple):
    product: str
    quantity: int
    price: Decimal

    def total(self):
        return self.price * self.quantity


class Order(NamedTuple):  # the Context
    customer: Customer
    cart: Sequence[LineItem]
    # promotion can be None or a callable object that takes
    # an argument of type Order and returns Decimal
    promotion: Optional[Callable[['Order'], Decimal]] = None

    def total(self) -> Decimal:
        totals = (item.total() for item in self.cart)
        return sum(totals, start=Decimal(0))

    def due(self) -> Decimal:
        if self.promotion is None:
            discount = Decimal(0)
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        return f"<Order total: {self.total():.2f} due: {self.due():.2f}>"


# There is no more Abstract class
# Each strategy is a function
def fidelity_promo(order: Order) -> Decimal:
    """5% discount for customers with 1000 or more fidelity points"""
    if order.customer.fidelity >= 1000:
        return order.total() * Decimal('0.05')
    return Decimal(0)


def bulk_item_promo(order: Order) -> Decimal:
    """10% discount for each LineItem with 20 or more units"""
    discount = Decimal(0)
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * Decimal('0.1')
    return discount


def large_order_promo(order: Order) -> Decimal:
    """7% discount for orders with 10 or more disctinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * Decimal('0.07')
    return Decimal(0)

In [10]:
joe = Customer(name='John Doe', fidelity=0)
ann = Customer(name='Ann Smith', fidelity=1100)

cart = (
    LineItem(product='banana', quantity=4, price=Decimal('.5')),
    LineItem(product='apple', quantity=10, price=Decimal('1.5')),
    LineItem(product='watermelon', quantity=5, price=Decimal(5)),
)

banana_cart = (
    LineItem(product='banana', quantity=30, price=Decimal('.5')),
    LineItem(product='apple', quantity=10, price=Decimal('1.5')),
)

long_cart = tuple(
    LineItem(product=str(sku), quantity=1, price=Decimal(1)) for sku in range(10)
)

In [11]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [12]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [13]:
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [14]:
Order(ann, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [15]:
Order(joe, long_cart, large_order_promo)

<Order total: 10.00 due: 9.30>

In [16]:
Order(ann, long_cart, large_order_promo)

<Order total: 10.00 due: 9.30>

<h3>Choosing the Best Strategy: a Simple Approach</h3>

The `best_promo` function applies all strategies and returns the biggest discount

In [17]:
PROMOS = [fidelity_promo, bulk_item_promo, large_order_promo]


def best_promo(order: Order) -> Decimal:
    """Compute the best discount available"""
    return max(promo(order) for promo in PROMOS)

In [18]:
Order(joe, cart, best_promo)

<Order total: 42.00 due: 42.00>

In [19]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

In [20]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [21]:
Order(ann, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [22]:
Order(joe, long_cart, best_promo)

<Order total: 10.00 due: 9.30>

In [23]:
Order(ann, long_cart, best_promo)

<Order total: 10.00 due: 9.30>

<h3>Search for Strategies in the Module</h3>

`globals()` returns a dictionary representing the current table of global symbologies. This is always the dictionary of the current module (within a function or method, it is the module where the function or method is defined, not the module from which it is called).

In [24]:
PROMOS = [
    promo
    for name, promo in globals().items()
    if name.endswith('_promo') and name != 'best_promo'
]


def best_promo(order: Order) -> Decimal:
    """Compute the best discount available"""
    return max(promo(order) for promo in PROMOS)

In [25]:
Order(joe, cart, best_promo)

<Order total: 42.00 due: 42.00>

In [26]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

In [27]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [28]:
Order(ann, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [29]:
Order(joe, long_cart, best_promo)

<Order total: 10.00 due: 9.30>

In [30]:
Order(ann, long_cart, best_promo)

<Order total: 10.00 due: 9.30>

<h2>Pattern Strategy, Complete with Decorator</h2>

The list `PROMOS` is filled out by the `@promotion` decorator

In [31]:
from decimal import Decimal
from typing import Callable, NamedTuple, Optional


class Customer(NamedTuple):
    name: str
    fidelity: int


class LineItem(NamedTuple):
    product: str
    quantity: int
    price: Decimal

    def total(self):
        return self.price * self.quantity


class Order(NamedTuple):
    customer: Customer
    cart: Sequence[LineItem]
    promotion: Optional[Callable[['Order'], Decimal]] = None

    def total(self) -> Decimal:
        totals = (item.total() for item in self.cart)
        return sum(totals, start=Decimal(0))

    def due(self) -> Decimal:
        if self.promotion is None:
            discount = Decimal(0)
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        return f"<Order total: {self.total():.2f} due: {self.due():.2f}>"


Promotion: Callable[[Order], Decimal]

# The list is global at module and is initially empty
PROMOS: list[Promotion] = []


# The promotion registration decorator returns the promo_func function unchanged,
# but adds it to PROMOS list
def promotion(promo: Promotion) -> Promotion:
    PROMOS.append(promo)
    return promo


# The best_promo function is unchanged because
# it depends only on the list of PROMOS
def best_promo(order: Order) -> Decimal:
    """Compute the best discount available"""
    return max(promo(order) for promo in PROMOS)


# All functions decorated by @promotion are added to PROMOS
@promotion
def fidelity(order: Order) -> Decimal:
    """5% discount for customers with 1000 or more fidelity points"""
    if order.customer.fidelity >= 1000:
        return order.total() * Decimal('0.05')
    return Decimal(0)


@promotion
def bulk_item(order: Order) -> Decimal:
    """10% discount for each LineItem with 20 or more units"""
    discount = Decimal(0)
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * Decimal('0.1')
    return discount


@promotion
def large_order(order: Order) -> Decimal:
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * Decimal('0.07')
    return Decimal(0)

In [32]:
Order(joe, cart, best_promo)

<Order total: 42.00 due: 42.00>

In [33]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

In [34]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [35]:
Order(ann, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [36]:
Order(joe, long_cart, best_promo)

<Order total: 10.00 due: 9.30>

In [37]:
Order(ann, long_cart, best_promo)

<Order total: 10.00 due: 9.30>

In [38]:
PROMOS

[<function __main__.fidelity(order: __main__.Order) -> decimal.Decimal>,
 <function __main__.bulk_item(order: __main__.Order) -> decimal.Decimal>,
 <function __main__.large_order(order: __main__.Order) -> decimal.Decimal>]

<h2>The Command Pattern</h2>

The Command pattern is a behavioral design pattern that encapsulates a request as an object, thereby allowing parameterization of clients with queues, requests, and operations. It enables the separation of the requester of an action from the object that performs the action, providing a layer of abstraction between the two.

In this example:</br>
The `Command` interface declares a method `execute()` which concrete command classes implement.</br>
Concrete command classes (`LightOnCommand`, `LightOffCommand`) encapsulate the action to be performed and reference the receiver (`Light`) that performs the action.</br>
The `Light` class represents the receiver and defines the actions (`turn_on`, `turn_off`) to be performed.</br>
The `RemoteControl` class serves as the invoker and is responsible for executing commands. It sets the command to be executed and invokes the `execute()` method when required.</br>
The `RemoteControl` object sets and executes the commands (`light_on`, `light_off`).

In [39]:
from abc import ABC, abstractmethod
from typing import Optional


# Define a command interface
class Command(ABC):
    @abstractmethod
    def execute(self) -> None:
        pass


# Define concrete command classes
class LightOnCommand(Command):
    def __init__(self, light: 'Light') -> None:
        self.light = light

    def execute(self) -> None:
        self.light.turn_on()


class LightOffCommand(Command):
    def __init__(self, light: 'Light') -> None:
        self.light = light

    def execute(self) -> None:
        self.light.turn_off()


# Define the receiver class
class Light:
    def turn_on(self) -> None:
        print("Light is on!")

    def turn_off(self) -> None:
        print("Light is off!")


# Define the invoker class
class RemoteControl:
    def __init__(self) -> None:
        self.command: Optional[Command] = None

    def set_command(self, command: Command) -> None:
        """Set the command to be execute by the remote control"""
        self.command = command

    def press_button(self) -> None:
        if self.command:
            self.command.execute()

In [40]:
light = Light()
light_on = LightOnCommand(light)
light_off = LightOffCommand(light)

remote_control = RemoteControl()

In [41]:
# Set and execute the commands
remote_control.set_command(light_on)
remote_control.press_button()

Light is on!


In [42]:
remote_control.set_command(light_off)
remote_control.press_button()

Light is off!
